In [ ]:
import sys
#!{sys.executable} -m pip install spacy
!{sys.executable} -m pip install gensim

In [1]:
import numpy as np
import pandas as pd
import string
import matplotlib.pyplot as plt
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
import pyLDAvis.sklearn
from pylab import bone, pcolor, colorbar, plot, show, rcParams, savefig
import warnings
import gensim
warnings.filterwarnings('ignore')

%matplotlib inline
import os


# spaCy based imports
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

#!{sys.executable} -m spacy download en_core_web_lg

/home/kslote/Epilepsy/virt/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [2]:
import mysql.connector as sql

def get_tweets():
    db_connection = sql.connect(host='localhost',
                database='gun',
                user='kslote',
                password='password')
    db_cursor = db_connection.cursor()
    db_cursor.execute('SELECT * FROM TWEETS ORDER BY RAND() LIMIT 10000;')
    table_rows = db_cursor.fetchall()
    df = pd.DataFrame(table_rows)

df = pd.read_csv("tweets.csv")
parser = English()
def spacy_tokenizer(sentence):
    mytokens = parser(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stopwords and word not in punctuations ]
    mytokens = " ".join([i for i in mytokens])
    return mytokens

In [3]:
filters=["gun violence", "gun rights", "second amendment", "assault weapons", " #2A ",
          "2nd amend", "guncontrol", "gun control", "mass shooting", "gun owner"]
b = ['gun control', 'universal background checks', 'assault weapon',
       '#2A', '2nd amendment', 'mass shooting', 'second amendment',
       'gun laws', 'gun violence', 'firearm', 'open carry', 'gun rights',
       'concealed carry',
       'gun control,gun laws,gun violence,mass shooting,assault weapon,firearm,universal background checks,second amendment,gun rights,open carry,concealed carry,constitutional carry,campus carry,2nd amendment,#2A',
       'constitutional carry', 'u:mnguncaucus', 'u:USACarry',
       'campus carry', "I've got the flu", 'u:RLSeikel', 'u:ericbaker2',
       'u:2Smithd', 'u:mcgoverntm', 'u:CeleryCaraway',
       'u:CathyCh51148231', 'u:ViolenceWonks', 'u:JamesTWalker3',
       'u:rickytwalsh', 'u:2ashrink', 'u:williamlharbuck',
       'u:NewtownAction', 'u:WalterNDiaz1', 'u:TaraMPaine']

df[2].unique()

array(['gun control', '#2A', 'gun laws', 'gun violence', 'assault weapon',
       'second amendment', '2nd amendment', 'universal background checks',
       'gun rights', 'firearm',
       'gun control,gun laws,gun violence,mass shooting,assault weapon,firearm,universal background checks,second amendment,gun rights,open carry,concealed carry,constitutional carry,campus carry,2nd amendment,#2A',
       'mass shooting', 'open carry', 'concealed carry',
       'constitutional carry', 'u:bugalubaby2011', 'campus carry',
       'u:wolfboy852001', "I've got the flu", 'u:melissacupcake',
       'u:AmericanWoman8', 'u:1donedream87', 'u:ShootersFayNC',
       'u:Soloyaker', 'u:AWRHawkins'], dtype=object)

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append(" ".join([token.lemma_ if token.lemma_ not in ['-PRON-'] else '' for token in doc if token.pos_ in allowed_postags]))
    return texts_out

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# Run in terminal: python3 -m spacy download en
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

# Do lemmatization keeping only Noun, Adj, Verb, Adverb


#data = df.loc[df[2] == 'gun control'][5].to_list()
data = df['text'].sample(10000).to_list()

data_words = list(sent_to_words(data))

print(data_words[:1])
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

vectorizer = CountVectorizer(analyzer='word',       
                             min_df=10,                        # minimum reqd occurences of a word 
                             stop_words='english',             # remove stop words
                             lowercase=True,                   # convert all words to lowercase
                             token_pattern='[a-zA-Z0-9]{3,}',  # num chars > 3
                             ngram_range=(1, 5),             # max number of uniq words
                            )

data_vectorized = vectorizer.fit_transform(data_lemmatized)

lda_model = LatentDirichletAllocation(n_components=2, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda_output = lda_model.fit_transform(data_vectorized)

print(lda_model)  # Model attributes
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        message = "Topic #%d: " % topic_idx
        message += ", ".join([feature_names[i]
                             for i in topic.argsort()[:-n_top_words - 1:-1]])
        print(message)


print_top_words(lda_model, vectorizer.get_feature_names(), 500)


[['rt', 'thedailyshow', 'according', 'to', 'fox', 'news', 'there', 'no', 'gun', 'in', 'america', 'gun', 'violence', 'epidemic', 'https', 'co', 'ifmk', 'ueuzp']]
LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
                          evaluate_every=-1, learning_decay=0.7,
                          learning_method='online', learning_offset=50.0,
                          max_doc_update_iter=100, max_iter=5,
                          mean_change_tol=0.001, n_components=2, n_jobs=None,
                          perp_tol=0.1, random_state=0, topic_word_prior=None,
                          total_samples=1000000.0, verbose=0)
Topic #0: gun, violence, shooting, gun violence, mass, mass shooting, carry, people, year, today, country, use, school, law, shoot, life, say, free, need, open, woman, know, police, victim, reason, break, stop, mental, end, amendment, second, border, tough, live, gun law, believe, thank, spree, issue, child, make, let, way, guy, allow, amp, conceal, ti

In [ ]:
Topic #0: gun, violence, shooting, gun violence, mass, mass shooting, carry, people, year, today, country, use, school, law, shoot, life, say, free, need, open, woman, know, police, victim, reason, break, stop, mental, end, amendment, second, border, tough, live, gun law, believe, thank, spree, issue, child, make, let, way, guy, allow, amp, conceal, time, research, health, come, talk, pro, instead, attack, just, look, safety, block, weapon, strong, video, family, fight, man, want, lose, great, conceal carry, crime, second amendment, arm, week, protect, control, state, terrorist, gun safety, happen, right, problem, nation, white, license, healthcare, blame, world, day, student, don, black, old, firearm, bring, stabbing, save, change, home, human, illness, mental illness, illegal, attempt, tough gun, kill, boy, water, place, new, year old, open border, store, address, game, charge, hear, open carry, stephenke, community, mental health, instead shooting, terrorist stabbing, spree tough, stabbing spree, terrorist stabbing spree instead shooting, instead shooting spree, instead shooting spree tough, spree tough gun, shooting spree, shooting spree tough, terrorist stabbing spree instead, stabbing spree instead shooting spree, reason terrorist stabbing, instead shooting spree tough gun, stabbing spree instead shooting, spree instead shooting, stephenke reason terrorist stabbing, shooting spree tough gun, spree instead shooting spree tough, stephenke reason terrorist, reason terrorist stabbing spree instead, spree instead, terrorist stabbing spree, stephenke reason terrorist stabbing spree, stephenke reason, stabbing spree instead, reason terrorist, spree instead shooting spree, reason terrorist stabbing spree, allow gun, end gun, camp, gunman, group, wake, candidate, president, speak, violence research, fear, violent, save life, fact, bear, people shoot, night, clean, hour, assault, entire, choice, effect, remember, concentration camp, concentration, end gun violence, bullet, act, prove, watch, advocate, tough crime, tough crime strong border, crime strong, crime strong border, strong border, tough crime strong, criminal, continue, buy, civilian, ago, plastic, father, offense, deal, pay, supporter, pass, free healthcare, forward, supremacist, white supremacist, begin, plastic free ocean, offense free, clean water concentration camp, healthcare plastic, free ocean clean water, concentration camp people, ocean clean, water concentration camp, offense free healthcare plastic, ocean, clean water concentration, free ocean clean water concentration, plastic free ocean clean, ocean clean water, water concentration, camp people, free ocean, healthcare plastic free, healthcare plastic free ocean clean, clean water concentration camp people, water concentration camp people, plastic free, clean water, free ocean clean, offense free healthcare, offense free healthcare plastic free, ocean clean water concentration, free healthcare plastic free, free healthcare plastic free ocean, healthcare plastic free ocean, free healthcare plastic, plastic free ocean clean water, ocean clean water concentration camp, innocent, hit, good, school shooting, arrest, safe, blood, number, help, drive, republican, opponent, look forward, choose, feel, office, commit, dead, video game, public, believe open, decade, stop mass, big, cop, proposal, victory, lobbyist, survivor, isn, step, look forward week, forward week opponent, forward week opponent believe open, week opponent, forward week opponent believe, week opponent believe, look forward week opponent, believe open border, opponent believe, week opponent believe open, week opponent believe open border, opponent believe open border, forward week, look forward week opponent believe, opponent believe open, control stop, zone, party, remove, win, daughter, ice, funding, die gun, flag, shit, stop mass shooting, inflict, need license, story, girl, soon, work, lead, fund, rank, bear arm, know gun, high, nation gun, war, red, record, really, free zone, refuse, month, law effect, standard, red flag, demand, prevention, fight second amendment, fight second, quickly, left, crisis, ask, people second, senator, floor, gun free, session, life gun, violence woman, cause, violence prevention, play, veteran, flag law, red flag law, guilty, carnage, buddy, threaten, yesterday, join, congratulation, write, amendment let, second amendment let, wonder, income, murder, plan, agree, live gun, loan, crime strong border fight second, border fight second amendment, crime strong border fight, strong border fight second, border fight, border fight second amendment let, fight second amendment let, border fight second, strong border fight, tough crime strong border fight, strong border fight second amendment, death, primary, make illegal, bullet hit people, second weapon, carnage quickly, gunman bullet hit people second, gunman bullet, weapon inflict carnage quickly, weapon inflict, bullet hit, weapon inflict carnage, bullet hit people second, second weapon inflict carnage quickly, inflict carnage quickly, second weapon inflict, people second weapon, hit people second weapon, gunman bullet hit, bullet hit people second weapon, hit people second, second weapon inflict carnage, hit people second weapon inflict, people second weapon inflict carnage, people second weapon inflict, hit people, inflict carnage, gunman bullet hit people, list, time year, morning, epidemic, gun lobbyist, care, reform, block gun, report, member, problem red flag, problem red, gun year, obvious, year ago, gun free zone, explain, cost, primary victory, drgregmurphy great, drgregmurphy great primary, congratulation drgregmurphy great, congratulation drgregmurphy, congratulation drgregmurphy great primary victory, great primary, drgregmurphy, great primary victory, drgregmurphy great primary victory, congratulation drgregmurphy great primary, high income country, world allow gun lobbyist write, income country world, lobbyist write nation, income country world allow gun, world allow, income country world allow, allow gun lobbyist write, allow gun lobbyist, high income country world, world allow gun lobbyist, write nation gun, country world, high income country world allow, nation gun law, income country, gun lobbyist write nation gun, country world allow gun lobbyist, gun lobbyist write nation, world allow gun, lobbyist write, lobbyist write nation gun law, write nation gun law, lobbyist write nation gun, gun lobbyist write, high income, allow gun lobbyist write nation, write nation, country world allow gun, country world allow, talk gun, gun safety proposal, safety proposal, car, hammer, tonight, invert, wife, promise, state obvious, gun safety proposal save life, obvious gun, gun safety proposal save, state obvious gun, proposal save life, obvious gun safety, state obvious gun safety, obvious gun safety proposal, safety proposal save, obvious gun safety proposal save, state obvious gun safety proposal, safety proposal save life, proposal save, woman act, violence woman act, worry, problem red flag law, guilty prove innocent invert, law guilty prove innocent invert, flag law guilty prove, guilty prove innocent invert standard, innocent invert, law guilty, innocent invert standard, red flag law guilty, prove innocent invert, law guilty prove innocent, red flag law guilty prove, prove innocent, guilty prove, guilty prove innocent, invert standard, prove innocent invert standard, flag law guilty prove innocent, flag law guilty, problem red flag law guilty
Topic #1: gun, law, control, gun control, amendment, people, right, second, weapon, second amendment, check, just, background, gun law, background check, assault, pass, want, ban, shoot, say, assault weapon, universal, universal background, need, kill, universal background check, state, think, make, support, protect, die, amp, work, time, high, know, long, government, tell, rifle, firearm, owner, flag, write, thing, murder, love, vote, assault weapon ban, weapon ban, total, red, common, shot, citizen, criminal, magazine, try, gun owner, capacity, red flag, new, house, day, military, let, high capacity, don, child, sense, flag law, change, capacity magazine, abide, high capacity magazine, amendment write, stand, person, little, city, control law, action, stop, shooting, people die, people shoot, red flag law, law abide, pistol, mean, strict, gun control law, limit, help, strong, actually, legislation, away, american, aware, gun right, common sense, run, use, win, defend, doesn, just aware, far, just aware amendment, just aware amendment write, write weapon pistol long, weapon pistol long rifle, write weapon pistol, aware amendment write weapon pistol, just aware amendment write weapon, weapon pistol, long rifle, amendment write weapon, weapon pistol long, aware amendment, amendment write weapon pistol long, aware amendment write weapon, pistol long, amendment write weapon pistol, write weapon, write weapon pistol long rifle, pistol long rifle, aware amendment write, sanctuary, big, wound, start, thousand, armed, month, allow, purchase, reinstate, prayer, strict gun, politician, happen, read, safety, case, weapon ban amp, amp limit high, reinstate assault weapon ban, people die reinstate assault, people die reinstate, amp limit high capacity magazine, capacity magazine pass, weapon ban amp limit high, die reinstate assault, ban amp limit high capacity, amp limit, assault weapon ban amp, reinstate assault, die reinstate assault weapon, amp limit high capacity, reinstate assault weapon, die reinstate assault weapon ban, assault weapon ban amp limit, limit high capacity magazine pass, magazine pass, limit high capacity magazine, limit high, people die reinstate assault weapon, ban amp, limit high capacity, ban amp limit high, ban amp limit, die reinstate, reinstate assault weapon ban amp, high capacity magazine pass, weapon ban amp limit, pass universal, reform, week, buy, crime, sell, question, require, care, pass universal background check, pass universal background, good, shooter, thought, amendment right, isn, kid, tweet, death, hold, common sense gun, sense gun, come, lot, pass gun, vet, point, police, majority, ask, follow, constitution, homicide, violence, ban assault, fact, leave, domestic, year, spend, kill shoot, attack, permit, shoot wound total shot, total shot, wound total shot, kill shoot wound total, kill shoot wound, shoot wound, wound total, kill shoot wound total shot, shoot wound total, gun safety, man, shoot kill, turn, push, include, history, saudi, democratic, love military, love military vet, military vet, thought prayer, answer, real, really, didn, confiscation, mentally, officer, fail, debate, declare, lose, strong gun, fight, create, life, gun reform, enact, lawmaker, constitutional, modern, strict gun law, bad, ban assault weapon, look, important, new gun, understand, hand, try pass, weekend, climate, liberal, send, hard, mentally ill, ill, job, loosen, white, plan, restriction, sweet little, murder spend month, spend month try pass universal, spend month, sweet little murder, spend month try pass, spend month try, month try pass, try pass universal background check, little murder spend month try, little murder spend, try pass universal background, month try, little murder, murder spend month try pass, murder spend month try, sweet little murder spend month, sweet, sweet little murder spend, murder spend, month try pass universal background, try pass universal, little murder spend month, month try pass universal, million, leader, refuse, legal, pair, sit, policy, federal, upset, sure, amendment person, big second, big second amendment, deranged, anti, modern upset pair, upset pair sneaker thousand child, epitome modern upset pair, sneaker, upset pair sneaker thousand, epitome modern, epitome modern upset, sneaker thousand, pair sneaker, thousand child, upset pair sneaker, upset pair, epitome, modern upset pair sneaker, epitome modern upset pair sneaker, sneaker thousand child, pair sneaker thousand, modern upset, modern upset pair sneaker thousand, pair sneaker thousand child, conservative, county, absolutely, big second amendment person, second amendment person, second amendment right, shot kill, shot kill shoot, shot kill shoot wound total, shot kill shoot wound, mayor, election, wrong, claim, lobby, article, ill deranged people, ill deranged people big, deranged people, mentally ill deranged people, ill deranged, mentally ill deranged, deranged people big second, people big second amendment person, people big, mentally ill deranged people big, people big second amendment, deranged people big, ill deranged people big second, people big second, deranged people big second amendment, sale, protect second, need gun, fuck, second amendment person work, amendment person work, person work, big second amendment person work, hear, want gun, table, enforce, nation, idea, journalist, member, protect second amendment, climate change, militia, able, terrorism, shoot wound total shot total, total homicide, shot total, wound total shot total, shot total homicide, total shot total homicide, wound total shot total homicide, total shot total, guarantee, weapon high, freedom, house pass, remember, mass, president, amendment sanctuary, training, problem, assault weapon high, left, mind, easy, trump, ban assault weapon high, effect, way, solution, sheriff, friend, large, public, measure, check ban, background check ban, minute, complete, gun death, talk, repeal, law want, background check red flag, check red, background check red, check red flag, check red flag law, background check red flag law, automatic, shoot gun, universal background check ban, https, pilot, border, make case, imagine, medium, excuse, endorsement, ignore, infringe, argument, current, violence hold, gun violence hold, reminder, terrorist, protest, sit table make, make case strong, make case strong gun, sit table make case, table make case, sit table, table make case strong, table make case strong gun, case strong, table make, sit table make case strong, case strong gun, exist, law assault, red flag law assault weapon, flag law assault, law assault weapon, flag law assault weapon, red flag law assault, sense gun law, common sense gun law, semi, shooting isn, amendment total, vet protect second, military vet protect, love military vet protect, second amendment total, vet protect, vet protect second amendment, vet protect second amendment total, protect second amendment total endorsement, military vet protect second amendment, protect second amendment total, amendment total endorsement, total endorsement, military vet protect second, love military vet protect second